In [1]:
PROJECT=!gcloud config get-value core/project 
#gcloud config list project --format "value(core.project)"
print("Your current GCP Project Name is: ", PROJECT)

Your current GCP Project Name is:  ['fml-gcp-project-01']


In [3]:
!gsutil mb gs://pedok-ml-gcp-vision-api

Creating gs://pedok-ml-gcp-vision-api/...
ServiceException: 409 A Cloud Storage bucket named 'pedok-ml-gcp-vision-api' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [4]:
!gsutil ls

gs://pedok-ml-gcp-vision-api/


In [5]:
!gcloud services enable vision.googleapis.com

In [6]:
!gcloud iam service-accounts create my-vision-sa --display-name "my vision service account"

Created service account [my-vision-sa].


In [7]:
!gcloud iam service-accounts keys create key.json --iam-account my-vision-sa@fml-gcp-project-01.iam.gserviceaccount.com
#!gcloud iam service-accounts keys create ~/key.json --iam-account my-vision-sa@{PROJECT_ID}.iam.gserviceaccount.com


created key [f03d7697ee42645571219830231c4f74fe285a5e] of type [json] as [key.json] for [my-vision-sa@fml-gcp-project-01.iam.gserviceaccount.com]


In [8]:
!set GOOGLE_APPLICATION_CREDENTIALS="D:\Portfolio\my-projects\GCP\key.json"

In [ ]:
!pip3 install -U pip google-cloud-vision

In [9]:
!gcloud projects describe fml-gcp-project-01

createTime: '2021-05-23T20:04:09.910Z'
lifecycleState: ACTIVE
name: fml-gcp-project-01
projectId: fml-gcp-project-01
projectNumber: '318144289060'


In [11]:
from google.cloud import vision

image_uri = 'gs://cloud-samples-data/vision/using_curl/shanghai.jpeg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.label_detection(image=image)

print('Labels (and confidence score):')
print('=' * 30)
for label in response.label_annotations:
    print(label.description, '(%.2f%%)' % (label.score*100.))

Labels (and confidence score):
Wheel (97.90%)
Tire (97.85%)
Bicycle (94.53%)
Photograph (94.24%)
Motor vehicle (91.37%)
Infrastructure (89.83%)
Bicycle wheel (85.65%)
Mode of transport (85.51%)
Asphalt (82.82%)
Umbrella (81.78%)


8. Perform Text Detection
Text detection performs Optical Character Recognition (OCR). It detects and extracts text within an image with support for a broad range of languages. It also features automatic language identification.

In this example, you will perform text detection on an image of an Otter Crossing. Copy the following snippet into your IPython session (or save locally as text_dectect.py):

In [12]:
from __future__ import print_function
from google.cloud import vision

image_uri = 'gs://cloud-vision-codelab/otter_crossing.jpg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.text_detection(image=image)

for text in response.text_annotations:
    print('=' * 30)
    print(text.description)
    vertices = ['(%s,%s)' % (v.x, v.y) for v in text.bounding_poly.vertices]
    print('bounds:', ",".join(vertices))

CAUTION
Otters crossing
for next 6 miles

bounds: (61,243),(251,243),(251,340),(61,340)
CAUTION
bounds: (75,245),(235,243),(235,269),(75,271)
Otters
bounds: (65,296),(140,297),(140,315),(65,314)
crossing
bounds: (151,295),(247,297),(247,318),(151,316)
for
bounds: (61,322),(94,322),(94,340),(61,340)
next
bounds: (106,322),(156,322),(156,340),(106,340)
6
bounds: (167,321),(180,321),(180,339),(167,339)
miles
bounds: (191,321),(251,321),(251,339),(191,339)


9. Perform Landmark Detection
Landmark detection detects popular natural and man-made structures within an image.

In [13]:
from __future__ import print_function
from google.cloud import vision

image_uri = 'gs://cloud-vision-codelab/eiffel_tower.jpg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.landmark_detection(image=image)

for landmark in response.landmark_annotations:
    print('=' * 30)
    print(landmark)

mid: "/g/120xtw6z"
description: "Trocad\303\251ro Gardens"
score: 0.9215528
bounding_poly {
  vertices {
    x: 339
    y: 54
  }
  vertices {
    x: 520
    y: 54
  }
  vertices {
    x: 520
    y: 353
  }
  vertices {
    x: 339
    y: 353
  }
}
locations {
  lat_lng {
    latitude: 48.861596299999995
    longitude: 2.2892823
  }
}

mid: "/m/02j81"
description: "Eiffel Tower"
score: 0.65712523
bounding_poly {
  vertices {
    x: 458
    y: 144
  }
  vertices {
    x: 495
    y: 144
  }
  vertices {
    x: 495
    y: 255
  }
  vertices {
    x: 458
    y: 255
  }
}
locations {
  lat_lng {
    latitude: 48.858461
    longitude: 2.294351
  }
}

mid: "/m/02j81"
description: "Eiffel Tower"
score: 0.65595245
bounding_poly {
  vertices {
    x: 448
    y: 72
  }
  vertices {
    x: 513
    y: 72
  }
  vertices {
    x: 513
    y: 281
  }
  vertices {
    x: 448
    y: 281
  }
}
locations {
  lat_lng {
    latitude: 48.875072
    longitude: 2.312622
  }
}



10. Perform Emotional Face Detection
Facial features detection detects multiple faces within an image along with the associated key facial attributes such as emotional state or wearing headwear.

In this example, you will detect the likelihood of emotional state from four different emotional likelihoods including: joy, anger, sorrow, and surprise.

To perform emotional face detection, copy the following Python code into your IPython session (or save locally as face_dectect.py):

In [14]:
from __future__ import print_function
from google.cloud import vision

uri_base = 'gs://cloud-vision-codelab'
pics = ('face_surprise.jpg', 'face_no_surprise.png')

client = vision.ImageAnnotatorClient()
image = vision.Image()

for pic in pics:
    image.source.image_uri = '%s/%s' % (uri_base, pic)
    response = client.face_detection(image=image)

    print('=' * 30)
    print('File:', pic)
    for face in response.face_annotations:
        likelihood = vision.Likelihood(face.surprise_likelihood)
        vertices = ['(%s,%s)' % (v.x, v.y) for v in face.bounding_poly.vertices]
        print('Face surprised:', likelihood.name)
        print('Face bounds:', ",".join(vertices))

File: face_surprise.jpg
Face surprised: LIKELY
Face bounds: (93,425),(520,425),(520,922),(93,922)
File: face_no_surprise.png
Face surprised: VERY_UNLIKELY
Face bounds: (120,0),(334,0),(334,198),(120,198)


https://github.com/googlecodelabs/cloud-vision-python


12. Additional Resources
In addition to the source code for the four examples you completed in this codelab, below are additional reading material as well as recommended exercises to augment your knowledge and use of the Vision API with Python.

Learn More
Cloud Vision API documentation: cloud.google.com/vision/docs
Cloud Vision API home page & live demo: cloud.google.com/vision
Vision API label detection/annotation: cloud.google.com/vision/docs/labels
Vision API facial feature recognition: cloud.google.com/vision/docs/detecting-faces
Vision API landmark detection: cloud.google.com/vision/docs/detecting-landmarks
Vision API optical character recognition (OCR): cloud.google.com/vision/docs/ocr
Vision API "Safe Search": cloud.google.com/vision/docs/detecting-safe-search
Vision API product/corporate logo detection: cloud.google.com/vision/docs/detecting-logos
Python on Google Cloud Platform: cloud.google.com/python
Google Cloud Python client: googlecloudplatform.github.io/google-cloud-python
Codelab open source repo: github.com/googlecodelabs/cloud-vision-

Additional Study
Now that you have some experience with the Vision API under your belt, below are some recommended exercises to further develop your skills:

You've built separate scripts demoing individual features of the Vision API. Combine at least 2 of them into another script. For example, add OCR/text recognition to the first script that performs label detection (label_detect.py). You may be surprised to find there is text on one of the hats in that image!

Instead of our random images available on Google Cloud Storage, write a script that uses one or more of your images on your local filesystem. Another similar exercise is to find images online (accessible via http://).
Same as #2, but with local images on your filesystem. Note that #2 may be an easier first step before doing this one with local files.

Try non-photographs to see how the API works with those.

Migrate some of the script functionality into a microservice hosted on Google Cloud Functions, or in a web app or mobile backend running on Google App Engine.

If you're ready to tackle that last suggestion but can't think of any ideas, here are a pair to get your gears going:

Analyze multiple images in a Cloud Storage bucket, a Google Drive folder (use the Drive API), or a directory on your local computer. Call the Vision API on each image, writing out data about each into a Google Sheet (use the Sheets API) or Excel spreadsheet. (NOTE: you may have to do some extra auth work as G Suite assets like Drive folders and Sheets spreadsheets generally belong to users, not service accounts.)

Some people Tweet images (phone screenshots) of other tweets where the text of the original can't be cut-n-pasted or otherwise analyzed. Use the Twitter API to retrieve the referring tweet, extract and pass the tweeted image to the Vision API to OCR the text out of those images, then call the Cloud Natural Language API to perform sentiment analysis (to determine whether it's positive or negative) and entity extraction (search for entities/proper nouns) on them. (This is optional for the text in the referring tweet.)
License

In [ ]:
from google.cloud import vision_v1

image_uri = 'gs://cloud-samples-data/vision/using_curl/shanghai.jpeg'

client = vision_V1.ImageAnnotatorClient()

source = {"image_uri": image_uri}
image = {"source": source}

features = [
        {"type_": vision_v1.Feature.Type.LABEL_DETECTION},
        {"type_": vision_v1.Feature.Type.IMAGE_PROPERTIES},
    ]

image = vision.Image()
image.source.image_uri = image_uri

response = client.label_detection(image=image)


print('Labels (and confidence score):')
print('=' * 30)
for label in response.label_annotations:
    print(label.description, '(%.2f%%)' % (label.score*100.))

Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy Unrestricted

!pip install --user apache-beam[gcp]==2.16.0 
!pip install --user tensorflow-transform==0.15.

BUCKET = 'qwiklabs-gcp-01-89767ecc5125'
PROJECT = 'qwiklabs-gcp-01-89767ecc5125'
REGION = 'us-central1'

import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

gcloud config set project $PROJECT
gcloud config set compute/region $REGION

%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi